In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
import random

In [2]:
batch_size=256
epochs=300
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=128
z_dim=128


In [3]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
def get_context_idx(N):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, 784), N)
    idx = torch.tensor(idx, device=device)
    return idx

In [5]:
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid

In [6]:
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y


In [7]:
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    
    x = torch.index_select(x_grid, dim=1, index=idx)
    
    x = x.expand(batch_size, -1, -1)
    
    return x

In [8]:
class CNP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(CNP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
        self.h_1 = nn.Linear(3, 256)
        self.h_2 = nn.Linear(256, 256)
        self.h_3 = nn.Linear(256, self.r_dim)

        self.r_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.r_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.z_dim + 2, 256)
        self.g_2 = nn.Linear(256,256)
        self.g_3 = nn.Linear(256,256)
        self.g_mu = nn.Linear(256, 1)
        self.g_sigma = nn.Linear(256, 1)

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def g(self, rep, x_target):
        r_et_x = torch.cat([rep, x_target], dim=2)
        input = F.relu(self.g_1(r_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        mu=self.g_mu(input)
        log_sigma=self.g_sigma(input)
        return mu,log_sigma

    def xy_to_r_params(self, x, y):
        x_y = torch.cat([x, y], dim=2)
        r_i = self.h(x_y)        
        r = self.aggregate(r_i)
        return r

    def forward(self, x_context, y_context):
        r_context = self.xy_to_r_params(x_context, y_context)  # (mu, logvar) of z
        r_expand = r_context.unsqueeze(1).expand(-1, 784, -1)
        # reconstruct the whole image including the provided context points
        #x_target size:batch_size*784*2
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        mu, log_sigma = self.g(r_expand, x_target)
    
        # Bound the variance
        sigma = 0.1 + 0.9 * F.softplus(log_sigma)

        return mu, sigma
        

In [9]:
def np_loss(mu,sigma, y):#, z_all, z_context

    if y is not None:
        log_p = -(y - mu)**2 / (2 * sigma**2) - torch.log(sigma)
    else:
        log_p = None
    return log_p

In [10]:
model = CNP(r_dim,z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
x_grid = generate_grid(28, 28)
os.makedirs("results_cnp/", exist_ok=True)

In [11]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        
        y_context = idx_to_y(context_idx, y_all)
        optimizer.zero_grad()
        
        mu,sigma= model(x_context, y_context)
        
        loss = -np_loss(mu,sigma, y_all).sum(dim=0).mean() #z_all, z_context
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [12]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)

            mu,sigma = model(x_context, y_context)
            test_loss += -np_loss(mu,sigma, y_all).sum(dim=0).mean().item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    recons1=[]
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(1):#in range(5) originally
                        mu,sigma= model(x_context, y_context)
                        recons.append(mu[:num_examples])
                        recons1.append(sigma[:num_examples])
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons1 = torch.cat(recons1).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons,recons1])
                    save_image(comparison.cpu(),
                               'results_cnp/ep_' + str(epoch) +
                               '_cps_' + str(N) + '.png', nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [13]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: -0.256158
Train Epoch: 1 [256/60000 (0%)]	Loss: -0.260415
Train Epoch: 1 [512/60000 (1%)]	Loss: -0.260819
Train Epoch: 1 [768/60000 (1%)]	Loss: -0.262615
Train Epoch: 1 [1024/60000 (2%)]	Loss: -0.265586


KeyboardInterrupt: 

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-6)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
#save the model
save_path="./results_cnp/model_param.pt"
torch.save(model.state_dict(), save_path)
